In [33]:
import pandas as pd

from heatmap.analysis import load_temperature_s, calculate_heating_load, linear_cop

In [39]:
heating_on_temperature = 17.0
min_temp = -5
max_temp = 20
min_cop = 3.5
max_cop = 7.5
annual_heat_kwh_consumption = 12000
temperature_fp = '../data/temperature_2020_2032.csv'

s_temperature = load_temperature_s(temperature_fp)
s_heating_load_kwh = calculate_heating_load(s_temperature, heating_on_temperature, annual_heat_kwh_consumption)
s_cop = s_temperature.apply(linear_cop)
s_elec_load_kwh = s_heating_load_kwh.divide(s_cop)

df = pd.concat([
    s_temperature,
    s_heating_load_kwh.rename('heating_load_kwh'),
    s_cop.rename('cop'),
    s_elec_load_kwh.rename('elec_load_kwh'),
], axis=1)

df

,temperature,heating_load_kwh,cop,elec_load_kwh
timestamp,,,,
2020-01-01 20:06:00,7.59,1.959130,5.5144,0.355275
2020-01-01 21:06:00,7.73,1.929982,5.5368,0.348574
2020-01-01 22:06:00,7.95,1.884179,5.5720,0.338151
2020-01-01 23:06:00,8.02,1.869605,5.5832,0.334863
2020-01-02 00:06:00,8.07,1.859195,5.5912,0.332522
...,...,...,...,...
2032-12-30 19:06:00,9.02,1.661409,5.7432,0.289283
2032-12-30 20:06:00,9.18,1.628097,5.7688,0.282225
2032-12-30 21:06:00,9.39,1.584376,5.8024,0.273055
